In [4]:
import asyncio
import sounddevice as sd
import soundfile as sf
import base64
import websockets.exceptions
import numpy as np
from gpiozero import RGBLED
from hume import HumeStreamClient
from hume.models.config import BurstConfig, ProsodyConfig

# define colors corresponding to each emotion (in RGB format)
emotion_colors = {
    'Anger': (1,0,0),         # red
    'Calmness': (0,1,0),     # green
    'Embarrassment': (1,1,0), # yellow
    'Excitement': (1,0.5,0),  # orange
    'Romance': (1,0,1),       # pink
    'Sadness': (0,0,1)        # blue
}

# Record audio
samplerate = 16000  # Hertz
duration = 3  # seconds
filename = 'output.wav'

# Function to encode audio (base64 encoding)
def encode_audio(filename):
    with open(filename, 'rb') as audio_file:
        return base64.b64encode(audio_file.read())

# Hume API interaction
async def main():
    client = HumeStreamClient("1Fuo6eVLpIj6ndhmC5VXllArH67eOcaSA0XLX3sHdU2SdEy5")
    burst_config = BurstConfig()
    prosody_config = ProsodyConfig()

    while True:
        try:
            async with client.connect([burst_config, prosody_config]) as socket:
                while True:
                    print(f"Recording for {duration} seconds...")
                    myrecording = sd.rec(int(samplerate * duration), samplerate=samplerate, channels=1, blocking=True)
                    print("Recording complete. Saving the audio as output.wav")
                    sf.write(filename, myrecording, samplerate)
                    encoded_audio = encode_audio(filename)
                    await socket.reset_stream()
                    result = await socket.send_bytes(encoded_audio)
                    print("Received response from Hume")

                    # interpret the emotion results
                    if not 'predictions' in result['prosody']:
                        print('no prediction')
                        continue
                    emotions = result['prosody']['predictions'][0]['emotions']
                    emotion_id = [4, 9, 22, 26, 38, 39]
                    emotion_values = [emotions[i]['score'] for i in emotion_id]
                    # print emotion names and values
                    for name, value in zip(emotion_colors.keys(), emotion_values):
                        print(f"{name}: {value}")
                    # find the highest emotion score
                    max_emotion_value = max(emotion_values)
                    # find the corresponding emotion name
                    max_emotion_name = list(emotion_colors.keys())[emotion_values.index(max_emotion_value)]
                    # # set the LED color to the color of the emotion with the highest score
                    # led.color = emotion_colors[max_emotion_name]

        except websockets.exceptions.ConnectionClosedError:
            print("Connection was closed unexpectedly. Trying to reconnect in 5 seconds...")
            await asyncio.sleep(3)


In [5]:
await main()

Recording for 3 seconds...
Recording complete. Saving the audio as output.wav
Received response from Hume
Anger: 0.0031839676667004824
Calmness: 0.014838169328868389
Embarrassment: 0.02772003598511219
Excitement: 0.5706271529197693
Romance: 0.010011066682636738
Sadness: 0.0399126261472702
Recording for 3 seconds...
Recording complete. Saving the audio as output.wav
Received response from Hume
Anger: 0.010305313393473625
Calmness: 0.016523364931344986
Embarrassment: 0.040537867695093155
Excitement: 0.04767446964979172
Romance: 0.008233345113694668
Sadness: 0.20773328840732574
Recording for 3 seconds...
Recording complete. Saving the audio as output.wav
Received response from Hume
Anger: 0.00563877122476697
Calmness: 0.03922724723815918
Embarrassment: 0.09245634078979492
Excitement: 0.26246750354766846
Romance: 0.008180513046681881
Sadness: 0.08717317879199982
Recording for 3 seconds...
Recording complete. Saving the audio as output.wav
Received response from Hume
Anger: 0.00576783623546

data transfer failed
Traceback (most recent call last):
  File "/Users/jessica.rahman/miniconda3/envs/mila/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 963, in transfer_data
    message = await self.read_message()
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jessica.rahman/miniconda3/envs/mila/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1033, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jessica.rahman/miniconda3/envs/mila/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1108, in read_data_frame
    frame = await self.read_frame(max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jessica.rahman/miniconda3/envs/mila/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1165, in read_frame
    frame = await Frame.read(
            ^^^^^^^^^^^^^^^^^
  File "/Users/jessica.rahman/min

Connection was closed unexpectedly. Trying to reconnect in 5 seconds...
Recording for 3 seconds...
Recording complete. Saving the audio as output.wav
Received response from Hume
Anger: 0.32479286193847656
Calmness: 0.02980823442339897
Embarrassment: 0.032058291137218475
Excitement: 0.022692788392305374
Romance: 0.004284316673874855
Sadness: 0.06286256015300751
Recording for 3 seconds...
Recording complete. Saving the audio as output.wav
Received response from Hume
Anger: 0.013410300947725773
Calmness: 0.08712071925401688
Embarrassment: 0.09350893646478653
Excitement: 0.0189761221408844
Romance: 0.02754281274974346
Sadness: 0.17925351858139038
Recording for 3 seconds...
Recording complete. Saving the audio as output.wav
Received response from Hume
Anger: 0.025924600660800934
Calmness: 0.08814828842878342
Embarrassment: 0.044064175337553024
Excitement: 0.02199378050863743
Romance: 0.017041170969605446
Sadness: 0.09970445185899734
Recording for 3 seconds...
Recording complete. Saving the 

CancelledError: 

In [2]:
# Create an event loop and run 'main'
loop = asyncio.get_event_loop()
loop.run_until_complete(main())




RuntimeError: This event loop is already running